# Advent of Code 2022, Day 5 Part 1
https://adventofcode.com/2022/day/5

In [9]:
from pathlib import Path
import numpy as np
import pandas as pd

In [10]:
input_filepath = Path('.\AoC_2022_5_input.txt')
if input_filepath.exists():
    print(f'{input_filepath} exists. Reading...')
else:
    print(f'{input_filepath} does not exist!')
with open(str(input_filepath), 'r') as fin:
    contents = fin.readlines()
print(f'Read {len(contents)} lines')

AoC_2022_5_input.txt exists. Reading...
Read 512 lines


In [11]:
def get_initial_stack_df(stack_lines):
    stack_width_plus_one = 4
    containers_per_row = 9
    
    stack = []
    for line in stack_lines:
        temp = line.strip('\n')
        row = [temp[i:i+stack_width_plus_one][1] \
               for i in stack_width_plus_one* np.arange(containers_per_row)]
        stack.append(row)
    
    df = pd.DataFrame(stack)
    df = df.replace(' ', np.NaN)

    # Move values to top of dataframe
    df = df.apply(lambda x: pd.Series(x.dropna().values))

    # Add some padding to the bottom
    pad_rows = 200
    df = pd.concat([df,pd.DataFrame(np.zeros([pad_rows, len(df.columns)])*np.nan)], 
                       ignore_index=True)

    return df

def parse_instruction(line):
    line_split = line.split()
    instruction = {'count': int(line_split[1]),
                  'from': int(line_split[3]),
                  'to': int(line_split[5])}
    return instruction

def pop_crates_per_instruction(df, instruction):
    crate_column = instruction['from']
    crate_depth = instruction['count']
    
    popped_crates = df.iloc[:crate_depth, crate_column-1]
    df[crate_column-1] = df[crate_column-1].shift(-crate_depth)
    
    return [df, popped_crates]

def drop_crates_in_reverse(df, instruction, crates):
    crate_column = instruction['to']
    crate_depth = instruction['count']
    reversed_creates = crates[::-1]
    df[crate_column-1] = df[crate_column-1].shift(crate_depth)
    df.iloc[:crate_depth, crate_column-1] = reversed_creates.values
    return df
    
    

In [12]:
# Convert text stack to a dataframe
df = get_initial_stack_df(contents[:8])
for line in contents[10:]:
    instruction = parse_instruction(line)
    df, crates_to_move = pop_crates_per_instruction(df, instruction)
    df = drop_crates_in_reverse(df, instruction, crates_to_move)
top = df.iloc[0].to_list()
print( 'Top row:\n' + ''.join(top))

Top row:
ZBDRNPMVH


# Part 2
https://adventofcode.com/2022/day/5#part2

In [13]:
def drop_crates(df, instruction, crates):
    crate_column = instruction['to']
    crate_depth = instruction['count']
    df[crate_column-1] = df[crate_column-1].shift(crate_depth)
    df.iloc[:crate_depth, crate_column-1] = crates.values
    return df
    

In [14]:
# Convert text stack to a dataframe
df = get_initial_stack_df(contents[:8])
for line in contents[10:]:
    instruction = parse_instruction(line)
    df, crates_to_move = pop_crates_per_instruction(df, instruction)
    df = drop_crates(df, instruction, crates_to_move)
top = df.iloc[0].to_list()
print( 'Top row:\n' + ''.join(top))

Top row:
WDLPFNNNB
